In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    database_name = '%s_upcache' % game
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{database_name}', connect_args={'connect_timeout': 10})

def get_quarter_time_boundaries(timestamp):
    q = {}
    q['start'] = 1648080000
    q['end'] = q['start'] + (86400 * 7 * 14) # 14 weeks, 13 weeks for the quarter, plus one trailing week
    while not (timestamp >= q['start'] and timestamp <= q['end']):
        q['start'] = q['end']
        q['end'] = q['start'] + (86400 * 7 * 14)
    return q

def get_game_content_okr_damage_sql(game, start_time, end_time):
    return """SELECT brr.day AS `day`,
                 1337274000 + 7*86400*(1+FLOOR((brr.day - 1337274000)/(7*86400)))  AS pvp_week_end,
                 IF(MOD(FLOOR((brr.day - 1337274000)/(7*86400)),2)=1,'ONP','Immortal')  AS pvp_week_type,
                 -SUM(brr.damage_iron_water_amount) AS `current_qtr_dmg`
                 FROM `%s_battles_risk_reward_daily_summary` AS brr
                 WHERE brr.day >= %d + 1*86400
                 AND brr.day < %d
                 AND brr.townhall_level >= 5
                 GROUP BY `day`;""" % (game, start_time, end_time)

def get_game_content_okr_dmg_dau_sql(game, start_time, end_time):
    return """SELECT ses.day AS `day`, SUM(ses.dau) AS `dau`
                     FROM %s_sessions_daily_summary AS ses
                     WHERE (ses.townhall_level >= 5 AND ses.day >= %d + 1*86400 AND ses.day < %d)
                     GROUP BY `day`;""" % (game, start_time, end_time)

def get_game_content_okr_durable_spend_sql(game, start_time, end_time):
    return """SELECT cur.day+1*86400 AS day,
              SUM(cur.total_price) AS `current_qtr_gamebucks`,
              (SELECT SUM(prev.total_price)
              FROM `tr_store_daily_summary` AS prev
              WHERE prev.day = cur.day - 90*86400
              AND prev.currency = 'gamebucks'
              AND prev.townhall_level >= 5
              AND prev.country_tier IN ('1','2')
              AND prev.category in ('research','resource_boost','crafting','building_upgrade','enhancement')) as previous_qtr_gamebucks
              FROM `tr_store_daily_summary` AS cur
              WHERE cur.day >= %d
              AND cur.day < %d+86400
              AND cur.currency = 'gamebucks'
              AND cur.townhall_level >= 5
              AND cur.country_tier IN ('1','2')
              AND cur.category in ('research','resource_boost','crafting','building_upgrade','enhancement')
              GROUP BY day
              ORDER BY day ASC LIMIT 1000;""" % (start_time, end_time)

def get_game_content_okr_spend_supply_sql(game, start_time, end_time):
    return """SELECT cur.time+1*86400 AS `day`,
              SUM(cur.total_amount) AS current_qtr_supply,
              (SELECT SUM(prev.total_amount)
              FROM `tr_active_player_resource_levels` AS prev
              WHERE (prev.time + 90*86400) BETWEEN cur.time - 43200 AND cur.time + 43200
              AND prev.townhall_level >= 5
              AND prev.country_tier IN ('1','2')
              AND prev.resource = 'gamebucks') as previous_qtr_supply
              FROM `tr_active_player_resource_levels` AS cur
              WHERE cur.time >= %d
              AND cur.time < %d + 86400
              AND cur.townhall_level >= 5
              AND cur.country_tier IN ('1','2')
              AND cur.resource = 'gamebucks'
              GROUP BY `day`
              ORDER BY `day` ASC LIMIT 1000;
              """ % (start_time, end_time)

time_now = int(time.time())
quarter_boundaries = get_quarter_time_boundaries(time_now)
start_time = quarter_boundaries['start']
end_time = quarter_boundaries['end']

# prepare game content OKR data
engine = get_engine('tr')
game_content_okr_dmg = {'day':[],'pvp_week_end':[], 'pvp_week_type':[],'current_qtr_dmg':[],'dau':[]}
game_content_okr_dmg_sql = get_game_content_okr_damage_sql('tr',start_time,end_time)
game_content_okr_dmg_data = pd.read_sql(game_content_okr_dmg_sql, engine)
for day in game_content_okr_dmg_data['day']:
    game_content_okr_dmg['day'].append(day)
for day in game_content_okr_dmg_data['pvp_week_end']:
    game_content_okr_dmg['pvp_week_end'].append(day)
for day in game_content_okr_dmg_data['pvp_week_type']:
    game_content_okr_dmg['pvp_week_type'].append(day)
for day in game_content_okr_dmg_data['current_qtr_dmg']:
    game_content_okr_dmg['current_qtr_dmg'].append(day)
game_content_okr_dmg_dau_sql = get_game_content_okr_dmg_dau_sql('tr',start_time,end_time)
game_content_okr_dmg_dau_data = pd.read_sql(game_content_okr_dmg_dau_sql, engine)
for day in game_content_okr_dmg_dau_data['dau']:
    game_content_okr_dmg['dau'].append(day)
    
game_content_okr_durable_spend_sql = get_game_content_okr_durable_spend_sql('tr',start_time,end_time)
game_content_okr_durable_supply_sql = get_game_content_okr_spend_supply_sql('tr',start_time,end_time)
game_content_okr_durable_spend_data = pd.read_sql(game_content_okr_durable_spend_sql, engine)
game_content_okr_durable_supply_data = pd.read_sql(game_content_okr_durable_supply_sql, engine)
game_content_okr_last_q_total_spend_sql = get_game_content_okr_durable_spend_sql('tr',start_time - (86400 * 7 * 14), end_time - (86400 * 7 * 14))
game_content_okr_last_q_total_supply_sql = get_game_content_okr_spend_supply_sql('tr',start_time - (86400 * 7 * 14), end_time - (86400 * 7 * 14))
game_content_okr_last_q_spend_data = pd.read_sql(game_content_okr_last_q_total_spend_sql, engine)
game_content_okr_last_q_supply_data = pd.read_sql(game_content_okr_last_q_total_supply_sql, engine)

game_content_okr_durable_spend = {'day':[],'cur_spend':[], 'prev_spend':[],'cur_supply':[],'prev_supply':[]}
for day in game_content_okr_durable_spend_data['day']:
    game_content_okr_durable_spend['day'].append(day)
for day in game_content_okr_durable_spend_data['current_qtr_gamebucks']:
    game_content_okr_durable_spend['cur_spend'].append(day)
for day in game_content_okr_durable_spend_data['previous_qtr_gamebucks']:
    game_content_okr_durable_spend['prev_spend'].append(day)
for day in game_content_okr_durable_supply_data['current_qtr_supply']:
    game_content_okr_durable_spend['cur_supply'].append(day)
for day in game_content_okr_durable_supply_data['previous_qtr_supply']:
    game_content_okr_durable_spend['prev_supply'].append(day)
game_content_okr_last_q_spend = {'day':[],'spend':[], 'supply':[]}
for day in game_content_okr_last_q_spend_data['day']:
    game_content_okr_last_q_spend['day'].append(day)
for day in game_content_okr_last_q_spend_data['current_qtr_gamebucks']:
    game_content_okr_last_q_spend['spend'].append(day)
for day in game_content_okr_last_q_supply_data['current_qtr_supply']:
    game_content_okr_last_q_spend['supply'].append(day)
    
    
print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))

Dashboard updated Sun, 24 Apr 2022 at 00:08:51 UTC


In [2]:
print('TR Cumulative Damage Suffered Per DAU (TOC L5+, x10000)')
boundaries = get_quarter_time_boundaries(time.time())
boundaries['start'] = boundaries['start'] + 86400*7
game_content_okr_total_dmg = {}
game_content_okr_weeks = []
for i, day in enumerate(game_content_okr_dmg['day']):
    week_int = int(game_content_okr_dmg['pvp_week_end'][i])
    if week_int not in game_content_okr_total_dmg:
        game_content_okr_total_dmg[week_int] = {'pvp_week_type':game_content_okr_dmg['pvp_week_type'][i], 'total':0}
    game_content_okr_total_dmg[week_int]['total'] += 0.0001 * game_content_okr_dmg['current_qtr_dmg'][i] / game_content_okr_dmg['dau'][i]
    if week_int not in game_content_okr_weeks:
        game_content_okr_weeks.append(week_int)
game_content_okr_total_onp = 0
game_content_okr_total_immortal = 0
header = "{0:12} {1:8} {2:10} {3:6}".format('Week', 'Damage', 'Type', 'Cumulative Damage')
print(header)
for week in game_content_okr_weeks:
    game_content_okr_week_data = game_content_okr_total_dmg[week]
    week_type = game_content_okr_week_data['pvp_week_type']
    this_week_total = math.ceil(game_content_okr_week_data['total'])
    output_total = 0
    if week_type == 'ONP':
        game_content_okr_total_onp += this_week_total
        output_total = game_content_okr_total_onp
    elif week_type == 'Immortal':
        game_content_okr_total_immortal += this_week_total
        output_total = game_content_okr_total_immortal
    formatted_date = datetime.datetime.utcfromtimestamp(week).strftime('%Y-%m-%d')
    if week < boundaries['start'] or week > boundaries['end']: continue
    output = "{0:12} {1:8} {2:10} {3:6}".format(formatted_date, str(this_week_total), week_type, str(output_total))
    print(output)
print('')
print('')


game_content_okr_last_q_total = 0
for i, day in enumerate(game_content_okr_last_q_spend['day']):
    game_content_okr_last_q_total += game_content_okr_last_q_spend['spend'][i] / game_content_okr_last_q_spend['supply'][i]
game_content_okr_last_q_daily_goal = 1.1 * game_content_okr_last_q_total / 13 / 7
print('TR Cumulative Durable Gamebuck Spend per Supply (TOC L5+) Tier 1/2')
header = "{0:12} {1:8} {2:10} {3:6}".format('Week', 'Target', 'Spend', 'Last Quarter')
print(header)
spend = 0
target = 0
prev_q_spend = 0
for i, day in enumerate(game_content_okr_durable_spend['day']):
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    if i < len(game_content_okr_durable_spend['cur_spend']) and i < len(game_content_okr_durable_spend['cur_supply']):
        spend += game_content_okr_durable_spend['cur_spend'][i] / game_content_okr_durable_spend['cur_supply'][i]
    if i < len(game_content_okr_durable_spend['prev_spend']) and i < len(game_content_okr_durable_spend['prev_supply']):
        prev_q_spend += game_content_okr_durable_spend['prev_spend'][i] / game_content_okr_durable_spend['prev_supply'][i]
    target += game_content_okr_last_q_daily_goal
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8} {2:10} {3:6}".format(output_date, str(round(target, 2)), str(round(spend, 2)), str(round(prev_q_spend, 2)))
        print(output_line)

TR Cumulative Damage Suffered Per DAU (TOC L5+, x10000)
Week         Damage   Type       Cumulative Damage
2022-03-31   1741     Immortal   1741  
2022-04-07   2275     ONP        2275  
2022-04-14   1904     Immortal   3645  
2022-04-21   1836     ONP        4111  
2022-04-28   268      Immortal   3913  


TR Cumulative Durable Gamebuck Spend per Supply (TOC L5+) Tier 1/2
Week         Target   Spend      Last Quarter
2022-03-31   0.17     0.13       0.15  
2022-04-07   0.34     0.29       0.28  
2022-04-14   0.5      0.45       0.41  
2022-04-21   0.67     0.63       0.56  
